In [6]:
import pandas as pd
import numpy  as np

import data file

In [7]:
df = pd.read_csv('data_new.csv')
df.describe()

,Label
count,329.000000
mean,0.589666
std,0.492644
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


split data

In [8]:
print("Non-cheat: ", len(df[df['Label']==0]))
print("Cheat: ", len(df[df['Label']==1]))

# diff = abs(len(df[df['Label']==0]) - len(df[df['Label']==1]))

grouped = df.groupby(df.Label)
df1 = grouped.get_group(1)
df2 = grouped.get_group(0)

df2_1 = df.iloc[:135,:]
df2_2 = df.iloc[135:194,:]

df1_train = df1.iloc[:111,:]
df2_train = df2_1.iloc[:111,:]

df1_test = df1.iloc[111:135,:]
df2_test = df2_1.iloc[111:135,:]

train = pd.concat([df1_train, df2_train])
test  = pd.concat([df1_test, df2_test, df2_2])

print((len(train), len(test)))

Non-cheat:  135
Cheat:  194
(222, 107)


parse data from string to numpy array

In [9]:
def parseData(series):
    final_series = []
    for ser in series:
        temp = []
        for x in "[],":
            ser = ser.replace(x, "")
        new_ser  = np.fromstring(ser, dtype=float, sep=" ")
        for i in range(0, len(new_ser), 3):
            chunk = [new_ser[i], new_ser[i + 1], new_ser[i + 2]]
            temp.append(chunk)
        final_series.append(temp)
    return np.array(final_series, dtype=object)

get the longest sequence of vector in dataset

In [10]:
max_length = 0
for lgaze in parseData(df['Left Gaze']):
    if len(lgaze) > max_length:
        max_length = len(lgaze)
        
max_length

1294

pad the dataset to the longest sequence size

In [11]:
import math

def padData(series, length):
    for idx, ser in enumerate(series):
        times = math.floor(length / len(ser))
        add = length % len(ser)
        
        temp = ser[::-1]
        for _ in range(1, times):
            series[idx] = np.append(series[idx], temp, axis=0)
            temp = temp[::-1]
        if add != 0:
            series[idx] = np.append(series[idx], temp[0:add], axis=0)
    return series

Conv1D -> AveragePooling1D -> Conv1D -> AveragePooling1D -> LSTM -> Dense

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, AveragePooling1D, LSTM
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf

def init_model():
    model = Sequential()
    model.add(Conv1D(filters=12, kernel_size=3, padding="same", activation='relu', input_shape=(max_length, 12)))
    model.add(AveragePooling1D(pool_size=2))
    model.add(Conv1D(filters=12, kernel_size=3, padding="same", activation='relu', input_shape=(max_length, 12)))
    model.add(AveragePooling1D(pool_size=2))
    
    model.add(LSTM(12))
    
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    
    model.add(Flatten())
    model.add(Dense(2, activation = 'softmax')) # We have only 2 classes: Non-cheat & Cheat
    
    model.compile(optimizer=adam,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [13]:
right_gaze_train = train["Right Gaze"]
left_gaze_train  = train["Left Gaze"]
right_head_train = train["Right HeadPose"]
left_head_train  = train["Left HeadPose"]

label_train = train['Label']

right_gaze_train, left_gaze_train, right_head_train, left_head_train = parseData(right_gaze_train), parseData(left_gaze_train), parseData(right_head_train), parseData(left_head_train)

initialize new data frame

In [14]:
left_head_train  = padData(left_head_train,  max_length)
right_head_train = padData(right_head_train, max_length)
left_gaze_train  = padData(left_gaze_train,  max_length)
right_gaze_train = padData(right_gaze_train, max_length)

In [15]:
label_train = np.array(label_train)

In [16]:
y_train = []
for i in range(len(label_train)):
    y_train.append(
        tf.convert_to_tensor(
            np.reshape(tf.keras.utils.to_categorical(label_train[i], num_classes=2), (1, 2))                 
                            )
             )
    
y_train = tf.convert_to_tensor(np.vstack(y_train), dtype=tf.float32)

In [17]:
x_train = []

for i in range(len(left_gaze_train)):
    x_train.append(tf.convert_to_tensor([
                                            np.hstack(
                                                        (left_gaze_train[i], right_gaze_train[i], left_head_train[i], right_head_train[i])
                                                     )
                                            ], dtype=tf.float32))



x_train = tf.convert_to_tensor(np.vstack(x_train), dtype=tf.float32)

print(x_train)

print(y_train)

tf.Tensor(
[[[  8.705  28.278  29.588 ...   1.558   2.732   3.145]
  [ -0.833  30.56  -30.571 ... -10.108   5.971 -11.741]
  [ -3.198  34.952 -35.098 ... -10.394   8.097 -13.176]
  ...
  [ -9.057  41.776 -42.746 ... -13.675  13.786 -19.418]
  [-16.372  28.36  -32.746 ... -12.991   8.778 -15.678]
  [-19.187  29.768 -35.416 ... -14.631   9.291 -17.332]]

 [[ 22.966  10.843  25.397 ...  30.359  -4.118  30.637]
  [  5.671  15.266  16.285 ...  18.896  -1.472  18.953]
  [  3.35   10.455  10.978 ...  -1.709   5.526  -5.784]
  ...
  [ -1.252  12.933 -12.994 ...   6.863   0.745   6.903]
  [ -3.019  14.169 -14.487 ...   9.237   2.032   9.458]
  [  7.474   5.718   9.41  ...   2.513   7.471   7.882]]

 [[ 27.556  25.147  37.306 ...  17.437  34.229  38.415]
  [ 20.462  44.667  49.131 ...   2.079  50.654  50.697]
  [ 15.712  58.805  60.868 ...  -3.587  62.252 -62.355]
  ...
  [ -0.803  48.756 -48.762 ... -18.131  46.641 -50.041]
  [ -0.776  49.995 -50.001 ... -18.296  47.122 -50.549]
  [ -4.184  49.

Train the model

In [18]:
model = init_model()
callbacks = [EarlyStopping('val_loss', patience=3)]
# model.fit(final_df, label, epochs=10, batch_size=128, validation_split=0.2, callbacks=callbacks)
model.fit(x_train, y_train, epochs=100, batch_size=128, callbacks=callbacks)

e:\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100
2/2 [==============================] - 8s 259ms/step - loss: 0.6759 - accuracy: 0.5946
Epoch 2/100
2/2 [==============================] - 0s 202ms/step - loss: 0.6285 - accuracy: 0.6937
Epoch 3/100
2/2 [==============================] - 0s 153ms/step - loss: 0.5926 - accuracy: 0.7342
Epoch 4/100
2/2 [==============================] - 0s 145ms/step - loss: 0.5606 - accuracy: 0.7793
Epoch 5/100
2/2 [==============================] - 0s 144ms/step - loss: 0.5359 - accuracy: 0.7973
Epoch 6/100
2/2 [==============================] - 0s 151ms/step - loss: 0.5267 - accuracy: 0.8108
Epoch 7/100
2/2 [==============================] - 0s 177ms/step - loss: 0.5072 - accuracy: 0.8108
Epoch 8/100
2/2 [==============================] - 0s 147ms/step - loss: 0.5005 - accuracy: 0.8108
Epoch 9/100
2/2 [==============================] - 0s 148ms/step - loss: 0.4958 - accuracy: 0.8108
Epoch 10/100
2/2 [==============================] - 0s 135ms/step - loss: 0.4891 - accuracy: 0.8108
Epoch 11/

In [19]:
for layer in model.layers:
    print(layer.output_shape)

(None, 1294, 12)
(None, 647, 12)
(None, 647, 12)
(None, 323, 12)
(None, 12)
(None, 12)
(None, 2)


Predict test set

In [20]:
right_gaze_test = test["Right Gaze"]
left_gaze_test  = test["Left Gaze"]
right_head_test = test["Right HeadPose"]
left_head_test  = test["Left HeadPose"]

label_test = test['Label']

right_gaze_test, left_gaze_test, right_head_test, left_head_test = parseData(right_gaze_test), parseData(left_gaze_test), parseData(right_head_test), parseData(left_head_test)

In [21]:
left_head_test  = padData(left_head_test,  max_length)
right_head_test = padData(right_head_test, max_length)
left_gaze_test  = padData(left_gaze_test,  max_length)
right_gaze_test = padData(right_gaze_test, max_length)

In [22]:
x_test = []

for i in range(len(left_gaze_test)):
    x_test.append(tf.convert_to_tensor([
                                            np.hstack(
                                                        (left_gaze_test[i], right_gaze_test[i], left_head_test[i], right_head_test[i])
                                                     )
                                            ], dtype=tf.float32))



x_test = tf.convert_to_tensor(np.vstack(x_test), dtype=tf.float32)

print(x_test)

tf.Tensor(
[[[ 51.537  30.285  59.777 ...  53.407   8.105  54.019]
  [ 40.51   37.984  55.533 ...  48.782   6.237  49.179]
  [ 26.477  32.743  42.108 ...  41.713  14.326  44.104]
  ...
  [  9.333  36.422  37.598 ...  32.341  15.118  35.7  ]
  [ 32.112  34.174  46.894 ...  47.997  12.341  49.559]
  [ 25.442  29.643  39.064 ...  41.91    9.913  43.066]]

 [[ 29.547  18.415  34.815 ...  44.61    5.529  44.951]
  [ 28.973  18.207  34.219 ...  43.922   4.782  44.182]
  [ 43.92   15.941  46.724 ...  48.461   1.124  48.474]
  ...
  [ 39.393   9.628  40.552 ...  41.659  -5.382  42.005]
  [ 35.149  10.188  36.595 ...  47.753  -4.01   47.921]
  [ 38.596   9.017  39.635 ...  49.893  -5.698  50.217]]

 [[-16.954  73.426 -75.358 ... -26.907  60.171 -65.913]
  [-22.978  82.974 -86.096 ... -32.98   67.892 -75.478]
  [-39.21   73.834 -83.599 ... -38.416  60.714 -71.847]
  ...
  [-30.736  50.905 -59.464 ... -21.113  18.457 -28.043]
  [-56.668  55.566 -79.366 ... -48.625  30.22  -57.251]
  [-43.283  75.

In [24]:
predictions = model.predict(x_test)

label_predictions = []

# convert from category to label
for prediction in predictions:
    if prediction[0] > prediction[1]:
        label_predictions.append(0)
    else: label_predictions.append(1)
    
label_predictions = np.array(label_predictions)
label_test = np.array(label_test)

print(label_predictions)
print(label_test)

# calc acc
true_count = 0
for idx, label in enumerate(label_predictions):
    if label == label_test[idx]:
        true_count += 1
        
print("Accuracy on test set: ", true_count/(len(label_test)))

4/4 [==============================] - 0s 48ms/step
[0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 1
 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0]
Accuracy on test set:  0.7383177570093458
